# Training the Model

In this notebook we make use the library `objdet` to create our own model using transfer learning, and train it in Google Colab to take advantage of their RAM and GPU.

## Google Colab

Mount your drive where the original data must be. It will be later converted to TFRecords. I decide to use Drive in order to have persistent storage, so I only have to upload and generate the TFRecords once. This speeds up the training process considerably.

In [ ]:
root_path = '/content/drive/My Drive/Colab Notebooks'

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
    print('User uploaded file "{name}" with length {length} bytes'.format(name=fn, length=len(uploaded[fn])))

In [ ]:
!pip install -r requirements.txt

In [ ]:
!ls /content/drive/'My Drive'/'Colab Notebooks'

Test if the GPU is available

In [ ]:
import tensorflow as tf
tf.test.gpu_device_name()

## Install ObjDet

In [ ]:
import objdet.tfapi as tfapi

In [ ]:
tfapi.install()

In [ ]:
# For some reason the notebook cannot find the variable PYTHONPATH
# set in tfapi.install(), but it can find it if the export is done
# manually before calling `jupyter notebook`. Strange behaviour.
import sys
sys.path.append("models/research")

# TODO: Fix. It works if the PYTHONPATH is exported before running the notebook
import objdet.tfrecords as tfrecords

## Download Model and Transfer Learning

Download the model for SSD300 on top of MobileNetV2 pretrained on MS COCO

In [ ]:
model_name = 'ssd_mobilenet_v2_coco_2018_03_29'
tfapi.download_model(model_name)

Generate the config files for our own model that only contains one class (i.e. hand)

In [ ]:
import os

classes = ['hand']
data_path = os.path.join(root_path, 'data', 'egohands')
tfapi.transfer_learning(classes, model_name, data_path)

## Create TFRecords

Create the TFRecords for all images in the `data` folder. This takes time and it requires around 13GB of space in disk.

In [ ]:
tfapi.create_tfrecords(data_path, model_name)

## Tensorboard

This code has been taken from https://www.dlology.com/blog/quick-guide-to-run-tensorboard-in-google-colab/

In [ ]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

In [ ]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [ ]:
get_ipython().system_raw('./ngrok http 6006 &')

In [ ]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

## Train and Save the Model

Now we are ready to train the model

In [ ]:
train_steps = 5000
eval_steps = 500
tfapi.train(model_name, train_steps, eval_steps)

After the training we can save the latest checkpoint as the trained model that will be later be loaded for prediction

In [ ]:
tfapi.save_model(model_name)

## Load Model and Predict

Load the trained model above

In [ ]:
model = tfapi.load_model(model_name)

In [ ]:
# TODO: Fix. It works if the PYTHONPATH is exported before running the notebook
import objdet.tfpredict as tfpredict

In [ ]:
import matplotlib.image
import numpy as np
from PIL import Image

img = np.array(Image.open('../data/test/CARDS_COURTYARD_B_T_frame_0484.jpg'))
tfpredict.predict(model, img)

In [ ]:
tfpredict.plot_prediction(model, img, model_name)